In [408]:
import numpy as np
import pandas as pd
from sklearn import datasets
import random
from sklearn.metrics import pairwise_distances
iris = datasets.load_iris()

In [384]:
data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
data['label'] = iris.target

In [385]:
data

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.1               3.5                1.4               0.2   
1                  4.9               3.0                1.4               0.2   
2                  4.7               3.2                1.3               0.2   
3                  4.6               3.1                1.5               0.2   
4                  5.0               3.6                1.4               0.2   
..                 ...               ...                ...               ...   
145                6.7               3.0                5.2               2.3   
146                6.3               2.5                5.0               1.9   
147                6.5               3.0                5.2               2.0   
148                6.2               3.4                5.4               2.3   
149                5.9               3.0                5.1               1.8   

     label  
0        0  
1        0  
2        0  
3        0  
4        0  
..     ...  
145      2  
146      2  
147      2  
148      2  
149      2  

[150 rows x 5 columns]

In [386]:
mainData = data.drop(["label"], axis=1).values
label = data["label"]
label

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: label, Length: 150, dtype: int64

In [387]:
mainData

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

so our chromosomes should have 150 cells (we ignore the last cell to tell us how many clusters we have becuase we recognize the number of clusters later)

In [388]:
data.head()

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   label  
0      0  
1      0  
2      0  
3      0  
4      0

In [389]:
data.isna().sum()


sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
label                0
dtype: int64

In [390]:
data.describe()

sepal length (cm)  sepal width (cm)  petal length (cm)  \
count         150.000000        150.000000         150.000000   
mean            5.843333          3.057333           3.758000   
std             0.828066          0.435866           1.765298   
min             4.300000          2.000000           1.000000   
25%             5.100000          2.800000           1.600000   
50%             5.800000          3.000000           4.350000   
75%             6.400000          3.300000           5.100000   
max             7.900000          4.400000           6.900000   

       petal width (cm)       label  
count        150.000000  150.000000  
mean           1.199333    1.000000  
std            0.762238    0.819232  
min            0.100000    0.000000  
25%            0.300000    0.000000  
50%            1.300000    1.000000  
75%            1.800000    2.000000  
max            2.500000    2.000000

In [391]:
data["label"].unique()


array([0, 1, 2])

finding the number of clusters we should define and  its 3

In [392]:
def objective_function(chromosome, data, num_clusters):
    clusters = [np.where(chromosome == i)[0] for i in range(0, num_clusters)]
    centroids = [np.mean(data[cluster], axis=0) for cluster in clusters]
    score = 0
    for i, cluster in enumerate(clusters):
        if len(cluster) > 0:
            distances = pairwise_distances(data[cluster], [centroids[i]], metric='euclidean')
            score += np.sum(distances ** 2)

    return score

In [393]:
def initialize_population(pop_size, num_points, num_clusters):
    population = []
    for _ in range(pop_size):
        chromosome = np.random.randint(0 , num_clusters , num_points)
        z=objective_function(chromosome,mainData,3)
        while(z>630):
            chromosome = np.random.randint(0 , num_clusters , num_points)
            z=objective_function(chromosome,mainData,3)    
        population.append(chromosome)
    return population

In [395]:
population=initialize_population(10,150,3)

In [396]:
population

[array([1, 1, 2, 0, 2, 0, 2, 2, 0, 1, 2, 0, 0, 0, 1, 2, 0, 2, 1, 0, 2, 2,
        0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 1, 0, 2, 2, 0, 2, 1, 2,
        2, 2, 0, 0, 0, 0, 2, 0, 1, 0, 1, 2, 0, 0, 0, 2, 0, 0, 1, 1, 1, 2,
        2, 2, 2, 1, 2, 1, 2, 0, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 0,
        2, 0, 2, 1, 2, 2, 0, 2, 0, 2, 0, 0, 1, 0, 2, 0, 0, 1, 0, 2, 0, 2,
        1, 1, 2, 0, 0, 1, 1, 2, 1, 1, 1, 0, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1,
        1, 1, 0, 0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 0, 2, 2, 0]),
 array([0, 0, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 1, 0, 0, 1, 1, 0,
        2, 1, 1, 2, 2, 2, 1, 1, 2, 0, 2, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 0,
        0, 1, 2, 1, 2, 2, 2, 2, 0, 0, 1, 2, 0, 0, 0, 1, 1, 0, 1, 0, 2, 1,
        0, 1, 1, 0, 2, 2, 0, 1, 0, 2, 2, 0, 1, 1, 1, 0, 1, 0, 0, 0, 2, 0,
        1, 0, 1, 1, 0, 2, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 2, 0, 0, 2, 2, 0,
        2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 2, 2, 0, 0, 2, 0, 0, 0, 0,
        1, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 1, 1

In [397]:
score = []
for chro in population:
    entry=objective_function(chro,mainData,3)
    score.append(entry)
    print(objective_function(chro,mainData,3))

620.9092791287976
602.2878824702927
626.791716641679
623.716552795031
617.1782704341967
625.0218519803182
624.3885708061
617.532152777778
628.7369423076916
626.17995062224


In [398]:
worstPoint=0
worstPointValue=630
def calculateWorst(score):
    worstPoint=np.argmax(score)
    worstPointValue=score[worstPoint]
calculateWorst(score)
worstPoint

#population[worstPoint]    

0

In [399]:
def mutation(population,score,chromosome):
    calculateWorst(score) 
    newChromosome=chromosome.copy()
    cell=np.random.randint(0,150)
    newChromosome[cell]=np.random.randint(0,3)
    #while(newChromosome!=chromosome):
    #   newChromosome[cell]=np.random.randint(0,3)
    newScore=objective_function(newChromosome,mainData,3) 
    if(newScore<worstPointValue):
        population[worstPoint]=newChromosome
        score[worstPoint]=newScore
        calculateWorst(score)    


In [400]:
crossover_point = np.random.randint(0,2)
crossover_point

0

In [401]:
def crossover(population,score,mainData,chro1,chro2):
    rand = np.random.randint(0,2)
    calculateWorst(score)
    if(rand==0):
        crossover_point = np.random.randint(0, 150)
        child1 = np.concatenate((chro1[:crossover_point], chro2[crossover_point:]))
        child2 = np.concatenate((chro2[:crossover_point], chro1[crossover_point:]))
        mutation(population,score,child1)
        mutation(population,score,child2)
        newScore1=objective_function(child1,mainData,3)
        newScore2=objective_function(child2,mainData,3)
        if(newScore1<worstPointValue):
            population[worstPoint]=child1
            score[worstPoint]=newScore1
            calculateWorst(score)
        if(newScore2<worstPointValue):
            population[worstPoint]=child2
            score[worstPoint]=newScore2
            calculateWorst(score)
    else:
        child1 = np.concatenate((chro1[:30], chro2[30:60],chro1[60:90],chro2[90:120],chro1[120:150]))
        child2 = np.concatenate((chro2[:30], chro1[30:60],chro2[60:90],chro1[90:120],chro2[120:150]))
        mutation(population,score,child1)
        mutation(population,score,child2)
        newScore1=objective_function(child1,mainData,3)
        newScore2=objective_function(child2,mainData,3)
        if(newScore1<worstPointValue):
            population[worstPoint]=child1
            score[worstPoint]=newScore1
            calculateWorst(score)
        if(newScore2<worstPointValue):
            population[worstPoint]=child2
            score[worstPoint]=newScore2
        calculateWorst(score)


In [402]:
def generic(mainData,iteration):
    population=initialize_population(50,150,3)
    score = []
    for chro in population:
        entry=objective_function(chro,mainData,3)
        score.append(entry)
    """
    for i in range(iteration):
        #parent1 , parent2 = random.sample(population, 2)
        crossover(mainData,parent1,parent2)
    """
    b=population[1]
    c=population[0]
    crossover(population,score,mainData,b,c)
    for i in range(iteration):
        parent1 , parent2 = random.sample(population, 2)
        crossover(population,score,mainData,parent1,parent2)

    bestPoint=np.argmin(score)
    population[bestPoint]
    return population[bestPoint]

 

In [403]:
sol=generic(mainData,100)

In [404]:
differences = sum(1 for a, b in zip(sol, label) if a != b)

In [405]:
differences

104

In [407]:
1-differences/150

0.30666666666666664